# Driver Eye Gaze Estimation 

Welcome to my notebook where i test different models i used for eye gaze estimation task and used at the beginning yolov8 for face detection

#### Models used for the task
Face detection :
- YoloV8

Eye Gaze Estimation:

- ResNet50
- MobileNetV2
- MobileNetV3
- SqueezeNet


## Test your webcam
will be needed to test the models below

In [1]:
import cv2

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    ret, img= cap.read()
    cv2.imshow('Webcam', img)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Face Detection
detecting user face as its our ROI which will be feeded into the models to boost accuracy

### YOLO V8

In [2]:
pip install ultralytics

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
from ultralytics import YOLO
model = YOLO("yolo-Weights/yolov8n.pt")

In [4]:
from ultralytics import YOLO
import cv2
import math 
# start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

# model
model = YOLO("C:/Users/yara/Desktop/dummyScript/yolov8n-face.pt")

# object classes
classNames = ["face", "no face"]
#eye gaze estimation classes
eyeGazeClasses=['Eyes Closed', 'Forward', 'Left Mirror', 'Radio', 'Rearview', 'Right Mirror', 'Shoulder', 'Speedometer']

while True:
    success, img = cap.read()
    results = model(img, stream=True)

    # coordinates
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # bounding box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

            # put box in cam
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # confidence
            confidence = math.ceil((box.conf[0]*100))/100
            print("Confidence --->",confidence)

            # class name
            cls = int(box.cls[0])
            print("Class name -->", classNames[cls])

            # object details
            org = [x1, y1]
            font = cv2.FONT_HERSHEY_SIMPLEX
            fontScale = 1
            color = (255, 0, 0)
            thickness = 2

            cv2.putText(img, classNames[cls], org, font, fontScale, color, thickness)

    cv2.imshow('Webcam', img)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ConnectionError: HTTPSConnectionPool(host='api.github.com', port=443): Max retries exceeded with url: /repos/ultralytics/assets/releases/tags/v8.1.0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B2948E2550>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

# Eye Gaze Estimation

In [5]:
# zones to be classified from
classes = ['Eyes Closed', 'Forward', 'Left Mirror', 'Radio', 'Rearview', 'Right Mirror', 'Shoulder', 'Speedometer']

## ResNet50

In [1]:
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
from ultralytics import YOLO
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

model_path="C:/Users/yara/Desktop/dummyScript/eyegaze.pt"
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(inplace=True),
    nn.Linear(512, 8),
    nn.Softmax(dim=1)
)

model = torch.load(model_path,map_location='cpu')
model.eval()

C:\Users\yara\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\yara\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
import cv2
from PIL import Image 
# Transformation for input to ResNet model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    success, frame = cap.read()

    # Convert frame to PIL Image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Preprocess the image
    input_image = transform(pil_image)
    input_tensor = input_image.unsqueeze(0)  # Add batch dimension

    # Run inference
    with torch.no_grad():
        output = model(input_tensor)

    # Get predicted label
    _, predicted_idx = torch.max(output, 1)
    predicted_label = classes[predicted_idx]

    # Display the predicted label on the frame
    cv2.putText(frame, predicted_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Webcam', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

## MobileNetV2

In [4]:
import torch
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

model_path_MNV2="C:/Users/yara/Desktop/dummyScript/MNV2_weights.h5"

model_MNV2 =torchvision.models.mobilenet_v2(pretrained=True)

for param in model_MNV2.parameters():
    param.requires_grad = False

model_MNV2.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=1280, out_features=8, bias=True),
            nn.Softmax(dim=1))

model_MNV2 = torch.load(model_path2,map_location='cpu')
model_MNV2.eval()

In [6]:
import cv2
from PIL import Image 
# Transformation for input to ResNet model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    success, frame = cap.read()

    # Convert frame to PIL Image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Preprocess the image
    input_image = transform(pil_image)
    input_tensor = input_image.unsqueeze(0)  # Add batch dimension

    # Run inference
    with torch.no_grad():
        output = model2(input_tensor)

    # Get predicted label
    _, predicted_idx = torch.max(output, 1)
    predicted_label = classes[predicted_idx]

    # Display the predicted label on the frame
    cv2.putText(frame, predicted_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Webcam', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

## MobileNetV3 Small

In [11]:
import torch
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

model_path_MNV3="C:/Users/yara/Desktop/dummyScript/MNV3_small_weights.h5"

model_MNV3 =torchvision.models.mobilenet_v3_small(pretrained=True)

for param in model_MNV3.parameters():
    param.requires_grad = False

# Define constants
NUM_CLASSES = 8
INPUT_FEATURES = 576
HIDDEN_FEATURES = 1024
OUTPUT_FEATURES = 1280

# Define the custom classifier
classifier = nn.Sequential(
    nn.Linear(INPUT_FEATURES, HIDDEN_FEATURES, bias=True),
    nn.Hardswish(),
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(HIDDEN_FEATURES, OUTPUT_FEATURES, bias=True),
    nn.Linear(OUTPUT_FEATURES, NUM_CLASSES, bias=True),  # Output layer with 8 classes
    nn.Softmax(dim=1)
)

# Replace the default classifier with the custom one
model_MNV3.classifier = classifier

model_MNV3 = torch.load(model_path_MNV3,map_location='cpu')

model_MNV3.eval()

In [12]:
import cv2
from PIL import Image 
# Transformation for input to ResNet model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    success, frame = cap.read()

    # Convert frame to PIL Image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Preprocess the image
    input_image = transform(pil_image)
    input_tensor = input_image.unsqueeze(0)  # Add batch dimension

    # Run inference
    with torch.no_grad():
        output = model_MNV3(input_tensor)

    # Get predicted label
    _, predicted_idx = torch.max(output, 1)
    predicted_label = classes[predicted_idx]

    # Display the predicted label on the frame
    cv2.putText(frame, predicted_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Webcam', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

## SqueezeNet

In [ ]:
import torch
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

model_path_sqz="C:/Users/yara/Desktop/dummyScript/MNV3_small_weights.h5"

model_sqz =torchvision.models.mobilenet_v3_small(pretrained=True)

for param in model_sqz.parameters():
    param.requires_grad = False



# Replace the default classifier with the custom one
model_sqz.classifier = classifier

model_sqz = torch.load(model_path_MNV3,map_location='cpu')

model_sqz.eval()

In [ ]:
import cv2
from PIL import Image 
# Transformation for input to ResNet model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Start webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

while True:
    success, frame = cap.read()

    # Convert frame to PIL Image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Preprocess the image
    input_image = transform(pil_image)
    input_tensor = input_image.unsqueeze(0)  # Add batch dimension

    # Run inference
    with torch.no_grad():
        output = model_sqz(input_tensor)

    # Get predicted label
    _, predicted_idx = torch.max(output, 1)
    predicted_label = classes[predicted_idx]

    # Display the predicted label on the frame
    cv2.putText(frame, predicted_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Webcam', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close OpenCV windows
cap.release()
cv2.destroyAllWindows()